# SARSIM DEMO
Demo Code to show how to use the SARSIM database
SARSIM includes tomographic data from the ESA campaigns:
BIOSAR 1, BIOSAR 2, TropiSAR, AfriSAR.
In addition, SARSIM also includes tomographic data acquired 
by JPL during the AfriSAR campaign.

SAR processing was carried out by Politecnico di Milano, DLR,
and University of Rennes. Please refer to the SARSIM manual for 
details about data processing.

The original script and related routines were developed by Mauro
Mariotti d'Alessandro and Stefano Tebaldini (PoliMi) using
Matlab 2015b on a 64 bit machine under Windows 10.

The demo code was converted to Python 3.6 by Guido Riembauer (ESTEC).
Required packages are numpy, matplotlib, os, math, gdal.

All Matlab and Python codes provided with SARSIM are intended for 
educational purposes only.

### Implemented Functions:
- data selection
- load SAR and ancillary data
- display loaded data
- generate tomographic vertical sections in SAR coordinates


### How to use this demo
The Jupyter notebook is split into two parts: 1) Preparation and 2) Processing.
In part 1), modules are imported and functions are defined. There is no user input 
required apart from running the sections. In the beginning of part 2), the user can 
then define the dataset to be analysed. 
<br> 
Run cells by pressing Ctrl+Enter or Shift+Enter to run a cell and proceed to the next one.
 

### Important: Set the path to the SARSIM database
Adapt the following string to your path to the SARSIM database and run the cell
by pressing Ctrl+Enter or Shift+Enter.

In [ ]:
sarsim_path = "D:\\SARSIM\\"

# 1) Preparation
Simply run the following sections to import packages/modules and to
define functions required for data loading and processing

### Import packages


In [ ]:
import numpy as np
import tifffile as tiff
import os
from math import ceil
%matplotlib notebook
import matplotlib.pyplot as plt

### Define functions to load SARSIM data

In [ ]:

def SARSIM_DATA_SELECTION(parent_folder,Operator, Site, Pol, Band, Sensor):
###############################################################################
# Operator = {DLR,ONERA,JPL}
# Site = {Lope, Remningstorp, Traunstein, Krycklan_SW, Krycklan_NE}
# Pol = {HH,HV,VH,VV}
# Band = {P,L}
# Sensor = {Airborne,Spaceborne}
###############################################################################
  
  ID = {"dataset":{}}
  ID["dataset"]["Operator"] = Operator
  ID["dataset"]["Site"] = Site
  ID["dataset"]["Pol"] = Pol
  ID["dataset"]["Band"] = Band
  ID["dataset"]["Sensor"] = Sensor
  ID["dtm_phase_removal"] = False
  
  # check if dtm phase removal is needed
  if ID["dataset"]["Operator"] == "DLR":
    if ID["dataset"]["Band"] == "L":
      if ID["dataset"]["Site"] == "Lope":
        ID["dtm_phase_removal"] = True
      elif ID["dataset"]["Site"] == "Traunstein":
        ID["dtm_phase_removal"] = True
  
  ID["parent_folder"] = parent_folder
  
  if Operator == "DLR":
    ID["operator_ID"] = 0
  elif Operator == "ONERA":
    ID["operator_ID"] = 1
  elif Operator == "JPL":
    ID["operator_ID"] = 2
  else:
    raise ValueError("Operator " + Operator + " does not exist in SARSIM. Available choices:{DLR,ONERA,JPL}")
  
  if Site == "Lope":
    ID["site_ID"] = 0
  elif Site == "Remningstorp":
    ID["site_ID"] = 1
  elif Site == "Traunstein":
    ID["site_ID"] = 2
  elif Site == "Krycklan_SW":
    ID["site_ID"] = 3
  elif Site == "Krycklan_NE":
    ID["site_ID"] = 4
  else:
    raise ValueError("Site " + Site + " does not exist in SARSIM. Available choices:{Lope,Remningstorp,Traunstein,Krycklan_SW,Krycklan_NE}")
    
  if Pol == "HH":
    ID["pol_ID"] = 0
  elif Pol == "HV":
    ID["pol_ID"] = 1
  elif Pol == "VH":
    ID["pol_ID"] = 2
  elif Pol == "VV":
    ID["pol_ID"] = 3
  else:
    raise ValueError("Polarization " + Pol + " does not exist in SARSIM. Available choices:{HH,HV,VH,VV}")
    
  if Band == "P":
    ID["band_ID"] = 0
  elif Band == "L":
    ID["band_ID"] = 1
  else:
    raise ValueError(Band + "-Band does not exist in SARSIM. Available choices:{P,L}")
  
  if Sensor == "Airborne":
    ID["sensor_ID"] = 0
  elif Sensor == "Spaceborne":
    ID["sensor_ID"] = 1
  else:
    raise ValueError("Sensor " + Sensor + " does not exist in SARSIM. Available choices:{Airborne,Spaceborne}")
    
  return ID
####################################################################################################

def SARSIM_param(ID):
  operator_ID = ID["operator_ID"]
  site_ID = ID["site_ID"]
  band_ID = ID["band_ID"]
  sensor_ID = ID["sensor_ID"]
  
  param = {}
  
  # Original airborne parameters
  if operator_ID == 0:
    if site_ID == 0:
      if band_ID == 0:                    # DLR Lope P
        param["N"] = 10
        param["x_ax_min"] = 0
        param["x_ax_max"] = 10938.4406
        param["dx"] = 0.90919
        param["Nx"] = 12032
        param["r_ax_min"] = 6536.3352
        param["r_ax_max"] = 11057.3403
        param["dr"] = 1.1989
        param["Nr"] = 3772
      elif band_ID == 1:                   # DLR Lope L
        param["N"] = 10
        param["x_ax_min"] = 0
        param["dx"] = 0.4546
        param["Nx"] = 24064
        param["x_ax_max"] = (param["Nx"]-1)*param["dx"]+param["x_ax_min"]
        param["r_ax_min"] = 6536.3352
        param["dr"] = 1.1989
        param["Nr"] = 3772
        param["r_ax_max"] = (param["Nr"]-1)*param["dr"]+param["r_ax_min"]
    elif site_ID == 1:                    # DLR Remningstorp P
      param["N"] = 14
      param["x_ax_min"] = 0
      param["x_ax_max"] = 5891.88
      param["dx"] = 0.74
      param["Nx"] = 7963
      param["r_ax_min"] = 4287.32
      param["r_ax_max"] = 6491.6723
      param["dr"] = 1.4985
      param["Nr"] = 1472
    elif site_ID == 2:                    # DLR Traunstein L
      param["N"] = 6
      param["x_ax_min"] = 0
      param["dx"] = 0.1951
      param["Nx"] = 43008
      param["x_ax_max"] = (param["Nx"]-1)*param["dx"]+param["x_ax_min"]
      param["r_ax_min"] = 5.9942e+03
      param["dr"] = 0.5994
      param["Nr"] = 4544
      param["r_ax_max"] = (param["Nr"]-1)*param["dr"]+param["r_ax_min"]
    elif site_ID == 3:                    # DLR Krycklan SW L
      param["N"] = 6
      param["x_ax_min"] = -4.260509397280971e+03
      param["dx"] = 0.469999024635399
      param["Nx"] = 19000
      param["x_ax_max"] = (param["Nx"]-1)*param["dx"]+param["x_ax_min"]
      param["r_ax_min"] = 4.292560000000000e+03
      param["dr"] = 1.498539999999593
      param["Nr"] = 1508
      param["r_ax_max"] = (param["Nr"]-1)*param["dr"]+param["r_ax_min"]
    elif site_ID == 4:                    # DLR Kycklan NE L
      param["N"] = 6
      param["x_ax_min"] = -3.540009242227824e+03
      param["dx"] = 0.4750
      param["Nx"] = 19186
      param["x_ax_max"] = (param["Nx"]-1)*param["dx"]+param["x_ax_min"]
      param["r_ax_min"] = 4.316536640000000e+03
      param["dr"] = 1.498540000000503
      param["Nr"] = 1468
      param["r_ax_max"] = (param["Nr"]-1)*param["dr"]+param["r_ax_min"]
  elif operator_ID == 1:
    if band_ID == 0:                      # ONERA Lope P
      param["N"] = 10
      param["x_ax_min"] = 0
      param["x_ax_max"] = 10369.2
      param["dx"] = 1.2
      param["Nx"] = 8642
      param["r_ax_min"] = 6600
      param["r_ax_max"] = 9714.5629
      param["dr"] = 2.3977
      param["Nr"] = 1300
    elif band_ID == 1:                    # ONERA Lope L
      param["N"] = 10
      param["x_ax_min"] = 0
      param["x_ax_max"] = 11502.4
      param["dx"] = 0.8
      param["Nx"] = 14379
      param["r_ax_min"] = 6900
      param["r_ax_max"] = 9539.2
      param["dr"] = 0.8
      param["Nr"] = 3300
  elif operator_ID == 2:                  # JPL Lope L
    param["N"] = 7
    param["x_ax_min"] = 0
    param["x_ax_max"] = 5999.4
    param["dx"] = 0.6
    param["Nx"] = 10000
    param["r_ax_min"] = 13148.9323
    param["r_ax_max"] = 18143.8078
    param["dr"] = 1.6655
    param["Nr"] = 3000
    
  param["x_axis"] = np.linspace(param["x_ax_min"], param["x_ax_max"], param["Nx"])
  
  # Spaceborne, same azimuth sampling
  if sensor_ID == 1:
    param["B"] = 6e6
    param["Az_res"] = 7
    if operator_ID == 0:
      if site_ID == 0:                  # DLR Lope P
        if band_ID == 0:
          param["r_ax_min"] =  763582.6471
          param["r_ax_max"] = 767420.1471
          param["dr"] = 12.5
          param["Nr"] = 308
          param["N"] = 7
      elif site_ID == 1:                 # DLR Remningstorp P
        param["r_ax_min"] = 764841.2522
        param["r_ax_max"] = 766728.7522
        param["dr"] = 12.5
        param["Nr"] = 152
        param["N"] = 7
    if operator_ID == 1:
      if band_ID == 0:                   # ONERA Lope P
        param["r_ax_min"] = 764058.4791
        param["r_ax_max"] = 766970.9791
        param["dr"] = 12.5
        param["Nr"] = 234
        param["N"] = 7
      else:                              # ONERA Lope L
        # Filtered, same parameters of the airborne
        param["B"] = 40e6
    elif operator_ID == 2:               # JPL
      param["N"] = 7
      param["x_ax_min"] = 0
      param["x_ax_max"] = 5999.4
      param["dx"] = 0.6
      param["Nx"] = 10000
      param["r_ax_min"] = 13148.9323
      param["r_ax_max"] = 18143.8078
      param["dr"] = 1.6655
      param["Nr"] = 3000
      param["B"] = 40e6
      
  param["r_axis"] = np.linspace(param["r_ax_min"], param["r_ax_max"], param["Nr"])

  return param  
###################################################################################################

def SARSIM_getstring(folder = None, operator_ID = None, band_ID = None, site_ID = None, pol_ID = None, pol_pair_ID = None, part_ind = None, type_ID = None):
  
  # generate strings to build paths 
  type_str_vect = ["SLC", "Kz", "DTM_SAR", "DTM_ground", "Tomo_Intensity", "Tomo_Coherence"]
  site_str_vect = ["Lope", "Remningstorp", "Traunstein", "Krycklan_SW", "Krycklan_NE"]
  operator_str_vect = ["DLR", "ONERA", "JPL"]
  band_str_vect = ["P", "L"]
  pol_str_vect = ["HH", "HV", "VH", "VV"]
  pol_pair_str_vect = ["HHHH","HVHV", "VHVH", "VVVV", "HHVV"]
  part_str_vect = ["Real", "Imag"]
  folders_str_vect = ["SLC_airborne\\", "Kz\\","DTM\\", "Tomo_airborne\\", "SLC_spaceborne\\", "Tomo_spaceborne\\"]

  if folder != None:
    string = folders_str_vect[folder]
  elif operator_ID != None:
    string = operator_str_vect[operator_ID]
  elif band_ID != None:
    string = band_str_vect[band_ID]
  elif site_ID != None:
    string = site_str_vect[site_ID]
  elif pol_ID != None:
    string = pol_str_vect[pol_ID]
  elif pol_pair_ID != None:
    string = pol_pair_str_vect[pol_pair_ID]
  elif part_ind != None:
    string = part_str_vect[part_ind]
  elif type_ID != None:
    string = type_str_vect[type_ID]

  return string
###################################################################################################

def LOAD_DTM(ID):
  
  parent_folder = ID["parent_folder"]
  sensor_ID = ID["sensor_ID"]
  operator_ID = ID["operator_ID"]
  site_ID = ID["site_ID"]
  band_ID = ID["band_ID"]
  
  if sensor_ID == 0 or \
  ((sensor_ID == 1) and (operator_ID == 1) and (band_ID == 1)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 0)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 2)) or \
  ((sensor_ID == 1) and (operator_ID == 2) and (band_ID == 1) and (site_ID == 0)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 3)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 4)):
  
    # Airborne
    
    curr_folder = parent_folder + SARSIM_getstring(folder = 2)
    dtm_filename = curr_folder + SARSIM_getstring(type_ID = 2) + "_"\
    + SARSIM_getstring(site_ID = site_ID) + "_" + SARSIM_getstring(operator_ID = operator_ID)\
    + "_" + SARSIM_getstring(band_ID = band_ID) + "_Band.tiff"
    print("Loading " + dtm_filename + "...")
    dtm = np.asarray(tiff.imread(dtm_filename))
    print("Loading done")
    
  else:
    
    # Spaceborne
    
    curr_folder = parent_folder + SARSIM_getstring(folder = 4)
    dtm_filename = curr_folder + SARSIM_getstring(type_ID = 2) + "_"\
    + SARSIM_getstring(site_ID = site_ID) + "_" + SARSIM_getstring(operator_ID = operator_ID)\
    + "_" + SARSIM_getstring(band_ID = band_ID) + "_Band.tiff"
    print("Loading " + dtm_filename + "...")
    dtm = np.asarray(tiff.imread(dtm_filename))
    print("Loading done")
    
  if ((operator_ID == 0) and (band_ID == 1) and (site_ID == 0)) or\
  ((operator_ID == 0) and (band_ID == 1) and site_ID == 2):
    dtm = np.rot90(np.flipud(dtm),-1)
    
  return dtm
####################################################################################################
def LOAD_KZ(ID, param):
  
  sensor_ID = ID["sensor_ID"]
  operator_ID = ID["operator_ID"]
  band_ID = ID["band_ID"]
  site_ID = ID["site_ID"]
  parent_folder = ID["parent_folder"]
  
  # If airborne or spaceborne filtered
  
  if sensor_ID == 0 or \
  ((sensor_ID == 1) and (operator_ID == 1) and (band_ID == 1)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 0)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 2)) or \
  ((sensor_ID == 1) and (operator_ID == 2) and (band_ID == 1) and (site_ID == 0)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 3)) or \
  ((sensor_ID == 1) and (operator_ID == 0) and (band_ID == 1) and (site_ID == 4)):
    
    curr_folder = parent_folder + SARSIM_getstring(folder = 1)
    if ID["dataset"]["Site"] == "Traunstein":
      curr_folder = parent_folder + "NOT_SHARED\\" + SARSIM_getstring(folder = 1)
      
    kz = np.zeros((param["Nr"], param["Nx"], param["N"]), dtype = np.float32)
    
    for n in np.arange(1,param["N"]+1):
      Kz_filename = curr_folder + SARSIM_getstring(type_ID = 1) + "_" + SARSIM_getstring(site_ID = site_ID)\
      + "_" + SARSIM_getstring(operator_ID = operator_ID) + "_" + SARSIM_getstring(band_ID = band_ID)\
      + "_Band_Pass_" + np.str(n) + ".tiff"
      print("Loading " + Kz_filename)
      # first dimension: range
      # second dimension: azimuth

      kz_temp = np.asarray(tiff.imread(Kz_filename))

      if ((operator_ID == 0) and (band_ID == 1) and (site_ID == 0)) or\
      ((operator_ID == 0) and (band_ID == 1) and (site_ID == 2)):
        kz[:,:,n-1] = np.rot90(np.flipud(kz_temp),-1)
      else:
        kz[:,:,n-1] = kz_temp
      print("Loading done")
  
  else: # spaceborne P-Band
  
    kz_vec = np.array([0,-0.044558,-0.08906,-0.13351,-0.17789,-0.22223,-0.2665])
    kz = np.zeros((param["Nr"], param["Nx"], param["N"]))
    for n in np.arange(0,param["N"]):
      kz[:,:,n] = np.ones((param["Nr"], param["Nx"])) * kz_vec[n]
    
  if ID["dataset"]["Site"] == "Remningstorp":
    if ID["dataset"]["Sensor"] == "Airborne":
      kz = -kz
      
  if ID["dataset"]["Site"] == "Krycklan_SW":
    kz = -kz
    
  if ID["dataset"]["Site"] == "Krycklan_NE":
    kz = -kz
    
  return kz
####################################################################################################

def LOAD_SLC_DATA(ID,param):
  
  parent_folder = ID["parent_folder"]
  sensor_ID = ID["sensor_ID"]
  operator_ID = ID["operator_ID"]
  site_ID = ID["site_ID"]
  band_ID = ID["band_ID"]
  pol_ID = ID["pol_ID"]
  
  # if required, switch HV/VH:
  if operator_ID == 0:
    if site_ID == 0:
      if pol_ID == 1:
        pol_ID = 2
        
  elif operator_ID == 1:
    if site_ID != 1:
      if band_ID == 0:
        if pol_ID == 1:
          pol_ID = 2
      elif band_ID == 1:
        if pol_ID == 2:
          pol_ID = 1
  
  elif operator_ID == 2:
    if site_ID != 1:
      if band_ID != 0:
        if pol_ID == 2:
          pol_ID = 1
  
  local_folder_str = SARSIM_getstring(folder = ((sensor_ID == 0) + (sensor_ID == 1)*5)-1)
  curr_folder = parent_folder + local_folder_str + SARSIM_getstring(pol_ID = pol_ID) + "\\"
  
  if ID["dataset"]["Site"] == "Traunstein":
    curr_folder = parent_folder + "NOT_SHARED\\" + local_folder_str + SARSIM_getstring(pol_ID = pol_ID) + "\\"
    
  I = np.zeros((param["Nr"],param["Nx"], param["N"]), dtype = "complex")
  for n in np.arange(1, param["N"]+1):
    for part_ind in np.arange(0,2):
      SLC_filename = curr_folder + SARSIM_getstring(type_ID = 0) + "_"  + SARSIM_getstring(site_ID = site_ID)\
      + "_" + SARSIM_getstring(operator_ID = operator_ID) + "_" + SARSIM_getstring(band_ID = band_ID)\
      + "_Band_" + SARSIM_getstring(pol_ID = pol_ID) + "_Pass_" + np.str(n)\
      + "_" + SARSIM_getstring(part_ind = part_ind) + "_part.tiff"
      print("Loading " + SLC_filename + "...")
      
      # first dimension: range
      # second dimension: azimuth
      
      slc_temp = np.asarray(tiff.imread(SLC_filename))

      
      if (site_ID == 2) or ((operator_ID == 0) and (site_ID == 0) and (band_ID == 1)):
        I[:,:,n-1] = I[:,:,n-1] + np.rot90(np.flipud(slc_temp),-1)*np.exp(1j*np.pi/2*(part_ind == 1))
        
      else:
        I[:,:, n-1] = I[:,:,n-1] + slc_temp *np.exp(1j*np.pi/2*(part_ind == 1))
      
      print("Loading done")

  return I
####################################################################################################

### Define TomoSAR processing functions

In [ ]:
def REMOVE_DTM_PHASE(ID, param, I, kz, dtm):
###############################################################################
# DTM PHASE REMOVAL
# THIS FUNCTION IS ONLY NECESSARY FOR THE FOLLOWING DATASETS:
# DLR L-BAND TRAUNSTEIN AIRBORNE
# DLR L-BAND TRAUNSTEIN SPACEBORNE
# DLR L-BAND LOPE AIRBORNE
# DLR L-BAND LOPE SPACEBORNE
###############################################################################
  if ID["dataset"]["Operator"] == "DLR":
    if ID["dataset"]["Band"] == "L":
      if ID["dataset"]["Site"] == "Lope":
        Z0 = 287.50483
      elif ID["dataset"]["Site"] == "Traunstein":
        Z0 = 666.16369
        
  for n in np.arange(0,param["N"]):
    I[:,:,n] = I[:,:,n]*np.exp(1j*kz[:,:,n]*(dtm-Z0))
  I = np.conjugate(I)
  return I 
###############################################################################


def Filter_Matrix(t_out,t_in,L):
###############################################################################
# Generates logic filtering matrix
#
# t_out = output axis in samples
# t_in  = input axis in samples
# L = half a window length in samples
###############################################################################
  t_in = t_in.T
  t_out = t_out.T
  T_in, T_out = np.meshgrid(t_in, t_out)
  F = np.abs(T_in-T_out)<(L+1)
  return F
###############################################################################


def Generate_Interferograms(I, r_ax, x_ax, Wr_m, Wx_m):
###############################################################################
# INPUT:
# I = stack of coregistered SLC images (range, azimuth, image)
# r_ax,x_ax = range and azimuth axes
# Wr_m, Wx_m = window size in range and azimuth, to be specified in the same 
# units as r_ax and x_ax
#
# OUTPUT:
# COV_4D = 4D array representing the complex correlation of any available 
# interferogram (range, azimuth, image 1, image 2)
# r_out_smpl, x_out_smpl = sample indices along range and azimuth
###############################################################################
  
  
  #############################################################################
  # DEFINITION OF WINDOW SIZE AND DOWN-SAMPLING
  Nr = np.shape(I)[0]
  Nx = np.shape(I)[1]
  N  = np.shape(I)[2]
  dr = r_ax[1] - r_ax[0]
  dx = x_ax[1] - x_ax[0]
  dr = np.abs(dr)
  dx = np.abs(dx)
  # filter and downsample along range
  Lr = ceil(Wr_m/2/dr)
  r_out_smpl = np.arange(Lr, Nr-Lr+1, ceil(Lr/2))
  if r_out_smpl.size == 0:
    r_out_smpl = int(np.round(Nr/2))
  Fr = Filter_Matrix(r_out_smpl, np.arange(1,Nr+1,1),Lr)
  Nr_out = np.shape(Fr)[0]
  # filter and downsample along azimuth
  Lx = ceil(Wx_m/2/dx)
  x_out_smpl = np.arange(Lx, Nx-Lx+1, ceil(Lx/2))
  if x_out_smpl.size == 0:
    x_out_smpl = int(np.round(Nx/2))
  Fx = Filter_Matrix(x_out_smpl, np.arange(1,Nx+1,1),Lx)
  Nx_out = np.shape(Fx)[0]
  #############################################################################
  
  #############################################################################
  # DATA COVARIANCE MATRIX
  COV_4D = np.ones((Nr_out, Nx_out, N, N), dtype = "complex")
  print("Interferogram generation...")
  for n in np.arange(0,N):
    In = I[:,:,n]
    for m in np.arange(n,N):
      Im = I[:,:,m]
      EX = Fr@(Im*np.conjugate(In))@Fx.conj().T
      COV_4D[:,:,n,m] = EX
      COV_4D[:,:,m,n] = np.conjugate(EX)
      # -1 for python indexing
  return COV_4D, r_out_smpl-1, x_out_smpl-1
###############################################################################



def Display_Interferograms(COV_4D, string):
  #############################################################################
  # Display intensity, coherence magnitude, and phase of all interferograms
  # INPUT:
  # COV_4D = 4D Interferogram array generated by function
  # Generate_Interferograms()
  #
  # string = string specified by the user
  # string = 'Intensity' => shows intensities in dB
  # string = 'Coherence' => shows coherence magnitude between 0 and 1
  # string = 'Phase' => shows interferogram phases between -pi and pi
  #############################################################################
  Nx_out = np.shape(COV_4D)[0]
  Ny_out = np.shape(COV_4D)[1]
  N      = np.shape(COV_4D)[3]
  DDX = np.zeros((N*Nx_out, N*Ny_out), dtype = "complex")
  for n in np.arange(0,N):
    ind_n_min = 0 + Nx_out*n
    ind_n_max = ind_n_min + Nx_out
    for m in np.arange(0,N):
      ind_m_min = 0 + Ny_out*m
      ind_m_max = ind_m_min + Ny_out
      DDX[ind_n_min:ind_n_max, ind_m_min:ind_m_max] = COV_4D[:,:,n,m]
      if string == "Coherence":
        DDX[ind_n_min:ind_n_max, ind_m_min:ind_m_max] =\
        COV_4D[:,:,n,m]/np.sqrt(np.real(COV_4D[:,:,n,n]*COV_4D[:,:,m,m]))
  
  plt.figure()
  ax = plt.subplot(111)
  if string == "Intensity":
    maps = ax.imshow(10*np.log10(abs(DDX)), cmap = "viridis", aspect = "auto")
    ax.set_title("InSAR Correlation [dB]")
  elif string == "Phase":
    maps = ax.imshow(np.angle(DDX), cmap = "viridis", vmin = -np.pi, vmax = np.pi, aspect = "auto")
    ax.set_title("InSAR Phase [rad]")
  elif string == "Coherence":
    maps = ax.imshow(abs(DDX), cmap = "viridis", vmin = 0, vmax = 1, aspect = "auto")
    ax.set_title("InSAR Coherence")
    
  n = np.arange(0, N*Ny_out, Ny_out)
  m = np.arange(0, N*Nx_out, Nx_out)
  
  for idx, mm in enumerate(m):
    ax.plot([0,N*Ny_out] ,[mm-0.5,mm-0.5], "k-", linewidth = 1.)
    ax.plot([n[idx]-0.5,n[idx]-0.5],[0,N*Nx_out],"k-", linewidth = 1.)
    
  ax.set_xlim(0,N*Ny_out)
  ax.set_ylim(N*Nx_out,0)
  ax.set_xticks([]) 
  ax.set_yticks([]) 
  plt.colorbar(maps)
  plt.tight_layout()
  pass
###############################################################################



def Correlation_Tomography(C,kz,z_ax_rel,delta_z_ref):
  #############################################################################
  # INPUT:
  # C = Matrix representing the correlations among all available InSAR pairs at
  # a specific rnage, azimuth loation. np.shape(C) = (N,N), where N is the 
  # number of flights
  #
  # kz = height-to-phase conversion factors. The Master image is assumed to be
  # the one for which kz = 0. np.shape(kz) = (N,1) or(1,N). where N is the
  # number of flights
  #
  # z_ax_rel = locations along the vertical direction at which the backscattered
  # power is to be calculated. np.shape(z_ax_rel) = (Nz,1) or (1,Nz), 
  # where Nz = len(z_ax_rel), the length of the vector
  #
  # delta_z_ref = scalar value representing reference forest height and used 
  # in the coherence interpolation kernel
  #
  # OUTPUT:
  # Tomo = Real-valued vector representing the estimated vertical distribution 
  # of backscattered power, calculated at the location specified in vector
  # z_ax_rel. np.shape(Tomo) = (Nz,1), where Nz = len(z_ax_rel), the length
  # of the vector
  #############################################################################
  kz = -kz # required by the conventions usd in SARSIM
  z_demod = delta_z_ref/2 # height used for coherence base-banding
  Nz = len(z_ax_rel) # number of vertical locations at which backscattering is
  # to be computed
  
  # looks for possible null values of kz other than the first one and sets them
  # to a small number
  kz[kz==0] = 1e-12*np.random.randn()
  kz[0] = 0
  
  # observed coherences
  coe = C[:,0]
  P0 = coe[0]
  if P0 == 0 or np.isnan(P0):
    Tomo = np.zeros(Nz)
  else:
    coe = coe/P0
    
    # Coherence interpolation: symmetrization
    kz_neg = np.where(kz<0)
    kz_pos = abs(kz)
    coe[kz_neg] = np.conjugate(coe[kz_neg])
    ind = np.argsort(kz_pos)
    coe = coe[ind]
    kz_pos = kz_pos[ind]
    coe_tri = np.concatenate((np.conjugate(np.flipud(coe[1:])), coe))
    kz_tri = np.concatenate((-np.flipud(kz_pos[1:]),kz_pos))
    
    # Coherence interpolation: interpolation on regular grid
    kz_max = np.max(abs(kz))
    N = np.shape(C)[0]
    kz_i = np.linspace(0,kz_max,N)
    kz_i = kz_i[1:] # ideal kz where coherence is measured
    N_pas = len(kz_i) # number of ideal measurements
    
    coe_i = np.interp(kz_i, kz_tri, coe_tri*np.exp(-1j*kz_tri*z_demod))\
    *np.exp(1j*kz_i*z_demod)
    
    # Tomography
    A = np.zeros((len(kz_i),len(z_ax_rel)), dtype = "complex")
    for idx, elem in enumerate(kz_i):
      for idx2, elem2 in enumerate(z_ax_rel):
        A[idx,idx2] = np.exp(1j*elem*elem2)
    n = np.arange(1,N_pas+1)
    weights = (N_pas+1-n)/(N_pas+1)
    coe_p = coe_i*weights
    Tomo = 1+ 2*np.real(A.conj().T@coe_p)
    Tomo = np.real(Tomo*(P0/(N_pas+1)))
    
  return Tomo
#############################################################################
  

# 2) Processing

## 2 A) Load and display data

### Data Selection
define the dataset you want to analyse by passing the
arguments in the following cell:<br>
<br>
ID = SARSIM_DATA_SELECTION(sarsim_path,'Operator','Site','Pol','Band','Sensor')<br>
Operator = {DLR,ONERA,JPL}<br>
Site = {Lope, Remningstorp, Traunstein, Krycklan_SW, Krycklan_NE}<br>
Pol = {HH,HV,VH,VV}<br>
Band = {P,L}<br>
Sensor = {Airborne,Spaceborne}<br>
<br>
Not every combination is available.

In [ ]:
# Data Selection
ID = SARSIM_DATA_SELECTION(sarsim_path,'JPL','Lope','HV','L','Airborne')

In [ ]:
# load data acquisition parameters
param = SARSIM_param(ID)

Load SAR and ancillary data. This may take a while depending on the number of acquisitions per site. Optionally you can reduce the number of acquisitions to be loaded in the following cell:

In [ ]:
# reduce the number of used passes
# param["N"] = 3

In [ ]:
# load digital terrain model (DTM) in SAR coordinates
dtm = LOAD_DTM(ID)
#local height-to-phase conversion factors
kz = LOAD_KZ(ID,param)
#load SAR Single-Look-Complex (SLC) data
I = LOAD_SLC_DATA(ID,param)

# remove dtm phase on DLR L-Band Lope and Traunstein data
if ID["dtm_phase_removal"]:
  I = REMOVE_DTM_PHASE(ID, param, I, kz, dtm)
ID["dtm_phase_removal"] = False

### Display loaded data

Average Intensity across all flights

In [ ]:
data_string = ID["dataset"]["Operator"] + " " + ID["dataset"]["Site"]  + " "\
+ ID["dataset"]["Pol"] + " " + ID["dataset"]["Band"] + " "\
+ ID["dataset"]["Sensor"]


Avg_intensity = np.mean(np.real(I*np.conjugate(I)),2)

# optimize contrast
vmin = np.percentile(10*np.log10(Avg_intensity[~np.isnan(Avg_intensity)]),5)
vmax = np.percentile(10*np.log10(Avg_intensity[~np.isnan(Avg_intensity)]),95)

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
extent = (param["x_ax_min"], param["x_ax_max"], param["r_ax_max"], param["r_ax_min"])
avg_map = ax1.imshow(10*np.log10(Avg_intensity), extent = extent, aspect = "equal", cmap = "viridis", vmin = vmin, vmax = vmax)
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax1.set_title("Average Intensity [dB] \n" + data_string)
ax1.set_xlabel("azimuth [m]")
ax1.set_ylabel("range [m]")
plt.colorbar(avg_map)
plt.tight_layout()
Avg_intensity = None


Interferogram phase between acquisitions n and m 

In [ ]:
n = 0
m = 1
Interf = I[:,:,n]*np.conjugate(I[:,:,m])
fig2 = plt.figure(2)
ax2 = fig2.add_subplot(111)
int_map = ax2.imshow(np.angle(Interf), extent = extent, aspect = "equal", cmap = "viridis")
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax2.set_title("Interferogram phase [rad] \n" + data_string)
ax2.set_xlabel("azimuth [m]")
ax2.set_ylabel("range [m]")
plt.colorbar(int_map)
plt.tight_layout()
Interf = None

Vertical resolution

In [ ]:
delta_kz = np.max(kz, axis = 2) - np.min(kz, axis = 2)
vertical_resolution_m = 2*np.pi/delta_kz
fig3 = plt.figure(3, figsize = (8,8))
ax3 = fig3.add_subplot(211)
res_map = ax3.imshow(vertical_resolution_m, extent = extent, aspect = "equal", cmap = "viridis")
ax3.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax3.set_title("Vertical resolution [m] \n" + data_string)
ax3.set_xlabel("azimuth [m]")
ax3.set_ylabel("range [m]")
plt.colorbar(res_map)
ax4 = fig3.add_subplot(212)
ax4.grid()
n, bins, patches = plt.hist(vertical_resolution_m.flatten()[np.isfinite(vertical_resolution_m.flatten())],100)
ax4.set_xlabel("Vertical resolution [m]")
ax4.set_ylabel("Occurences")
ax4.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tight_layout()
delta_kz, vertical_resolution_m = None,None

## 2 B) Generate tomographic vertical sections in SAR coordinates

### Interferogram generation

In [ ]:
Nr = np.shape(I)[0]
Nx = np.shape(I)[1]
N  = np.shape(I)[2]
if Nx > 11e3:               # crop data to reduce the computational burden
  ind_crop = np.arange(2000,7000,1)
  I = I[:,ind_crop,:]
  kz = kz[:,ind_crop,:]
  dtm = dtm[:,ind_crop]
  Nr = np.shape(I)[0]
  Nx = np.shape(I)[1]
  N  = np.shape(I)[2]
  
elif ID["site_ID"] == 1: # Crop NaN edges at Remningstorp
  ind_crop = np.arange(155,7795,1)
  I = I[:,ind_crop,:]
  kz = kz[:,ind_crop,:]
  dtm = dtm[:,ind_crop]
  Nr = np.shape(I)[0]
  Nx = np.shape(I)[1]
  N  = np.shape(I)[2]
  
# image coordinates
x_ax = param["x_axis"]
r_ax = param["r_axis"]
# window size
Wx_m = 20
Wr_m = 20
# Interferogram generation
COV_4D, r_out_smpl, x_out_smpl = Generate_Interferograms(I,r_ax,x_ax,Wr_m,Wx_m)
Nr_out = np.shape(COV_4D)[0]
Nx_out = np.shape(COV_4D)[1]
r_out_m = r_ax[r_out_smpl]
x_out_m = x_ax[x_out_smpl]
Kz_out = kz[r_out_smpl,:,:][:,x_out_smpl,:]
dtm_out = dtm[r_out_smpl,:][:,x_out_smpl]
Display_Interferograms(COV_4D, "Intensity")
Display_Interferograms(COV_4D, "Phase")
Display_Interferograms(COV_4D, "Coherence")

### Generation of a vertical section at a fixed azimuth position

In [ ]:
x0 = int(np.round(Nx_out/2,0)) # azimuth bin at which the vertical profile is computed
# Display section position over intensity
fig7 = plt.figure(7)
ax8 = fig7.add_subplot(111)
vmin = 10*np.log10(np.percentile(abs(COV_4D[:,:,0,0])[~np.isnan(COV_4D[:,:,0,0])], 2))
vmax = 10*np.log10(np.percentile(abs(COV_4D[:,:,0,0])[~np.isnan(COV_4D[:,:,0,0])], 98))

extent = (np.min(x_out_m), np.max(x_out_m), np.max(r_out_m), np.min(r_out_m))
map1 = ax8.imshow(10*np.log10(abs(COV_4D[:,:,0,0])), extent = extent, aspect = "equal", cmap = "viridis", vmin = vmin, vmax = vmax)
ax8.plot([x_out_m[x0], x_out_m[x0]],[r_out_m[0],r_out_m[-1]],"k-")
ax8.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax8.set_title("Average Intensity [dB]\n" + data_string)
ax8.set_xlabel("azimuth [m]")
ax8.set_ylabel("range [m]")
plt.colorbar(map1)

# Vertical axis w.r.t drm
dz = 1 # sampling along the direction in meters
z_ax_rel = np.arange(-20,71,dz) # height bins w.r.t to dtm 
Nz = len(z_ax_rel) # number of height bins

# Tomographic focusing method:
# tomo_processing_method == 1 <=> coherent tomography bases on all available
# Interferograms
# tomo_processing_method == 2 <=> Correlation tomography based on interferograms
# formed with a common master
tomo_processing_method = 2
if tomo_processing_method == 1:
  met_strg = "Coherent Tomography"
elif tomo_processing_method == 2:
  met_strg = "Correlation Tomography"
  
# Tomographic processing
Tomo = np.zeros((Nr_out,Nz))
for r in np.arange(0,Nr_out): # all range bins
  x = x0                    # selected azimuth bin ######### AT THE END: REMOVE -1 FOR TESTING!
  kz_loc = Kz_out[r,x,:] # local kz
  C = COV_4D[r,x,:,:] # local covariance matrix
  C[np.isnan(C)] = 0
  
  if False:                       # enforces power stationarity (optional)
    e = np.diag(C)
    P_ref = np.mean(e)
    e = np.diag(1./np.sqrt(e))
    C = P_ref*e@C@e
  
  if np.sum(C[np.isnan(C)]) == 0:
    if tomo_processing_method == 1: # Beamforming
      fase = np.zeros((len(kz_loc),len(z_ax_rel)))
      for idx,elem in enumerate(kz_loc):
        for idx2,elem2 in enumerate(z_ax_rel):
          fase[idx,idx2] = elem*elem2
      W = np.exp(-1j*fase)
      Tomo_rx = np.real(np.diag(W.conj().T@C@W))/N**2
    elif tomo_processing_method == 2: # Correlation Tomograpy
      delta_z_ref = 45
      Tomo_rx = Correlation_Tomography(C,kz_loc,z_ax_rel,delta_z_ref)
      Tomo_rx[Tomo_rx<0] = 0
    Tomo[r,:] = Tomo_rx

### Display vertical section

In [ ]:
extent = (np.min(r_out_m),np.max(r_out_m),np.max(z_ax_rel),np.min(z_ax_rel))
fig8 = plt.figure(8)
ax9 = fig8.add_subplot(211)
section1 = ax9.imshow(Tomo.T, extent = extent, cmap = "viridis", aspect = "auto")
plt.gca().invert_yaxis()
ax9.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax9.set_title("Vertical Section " + data_string + ' ' + met_strg)
ax9.set_xlabel("range [m]")
ax9.set_ylabel("height w.r.t DTM")
plt.colorbar(section1)

# Normalize by total intensity
norm_function = np.sum(Tomo,1)
Tomo_norm = Tomo.T/(np.ones((Nz,1))*norm_function)

# set the colorscale to optimize contrast
vmax = np.percentile(Tomo_norm[~np.isnan(Tomo_norm)],98)

ax10 = fig8.add_subplot(212)
section2 = ax10.imshow(Tomo_norm, extent = extent, cmap = "viridis", aspect = "auto", vmin = 0, vmax = vmax) 
plt.gca().invert_yaxis()
ax10.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax10.set_title("Normalized vertical Section " + data_string + ' ' + met_strg)
ax10.set_xlabel("range [m]")
ax10.set_ylabel("height w.r.t DTM")
plt.colorbar(section2)
plt.tight_layout()

## 2 C) Ground notching
Generate ground notching in SAR coordinates

In [ ]:
# Choice of the image pair
index_1 = 2
index_2 = 4
# Ground notched image
I_notch = I[:,:,index_1] - I[:,:,index_2]
# Vertical resolution
pho_z = 2*np.pi/(kz[:,:,index_1] - kz[:,:,index_2])
# Average intensity 
Avg_int = np.mean(I[:,:,[index_1,index_2]]*np.conjugate(I[:,:,[index_1,index_2]]),axis = 2)


Average Intensity before ground notching

In [ ]:
# optimize contrast
vmin = np.percentile(10*np.log10(np.real(Avg_int[~np.isnan(Avg_int)])),2)
vmax = np.percentile(10*np.log10(np.real(Avg_int[~np.isnan(Avg_int)])),98)


fig9 = plt.figure(9)
ax11 = fig9.add_subplot(111)
AvgI_map = ax11.imshow(10*np.log10(np.real(Avg_int)), extent = extent, aspect = "auto", cmap = "viridis", vmin = vmin, vmax = vmax)
ax11.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax11.set_title("Average Intensity [dB] \nimage pair " + np.str(index_1+1) + "," + np.str(index_2+1))
ax11.set_xlabel("azimuth [m]")
ax11.set_ylabel("range [m]")
plt.colorbar(AvgI_map)
plt.tight_layout()

Intensity after ground notching

In [ ]:
fig10 = plt.figure(10)
ax12 = fig10.add_subplot(111)
extent = (param["x_ax_min"], param["x_ax_max"], param["r_ax_max"], param["r_ax_min"])
Inotch_map = ax12.imshow(10*np.log10(np.real(I_notch*np.conjugate(I_notch))), extent = extent, aspect = "auto", cmap = "viridis", vmin = vmin, vmax = vmax)
ax12.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax12.set_title("Ground notched Intensity [dB] \nimage pair " + np.str(index_1+1) + "," + np.str(index_2+1))
ax12.set_xlabel("azimuth [m]")
ax12.set_ylabel("range [m]")
plt.colorbar(Inotch_map)
plt.tight_layout()

Vertical resolution

In [ ]:
fig11 = plt.figure(11)
ax13 = fig11.add_subplot(111)
pho_map = ax13.imshow(np.abs(pho_z), extent = extent, aspect = "auto", cmap = "viridis")
ax13.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax13.set_title("Vertical resolution [m] \nimage pair " + np.str(index_1+1) + "," + np.str(index_2+1))
ax13.set_xlabel("azimuth [m]")
ax13.set_ylabel("range [m]")
plt.colorbar(pho_map)
plt.tight_layout()